# Movie Recommender

GroupLens Research has collected and made available rating data sets from the MovieLens web site (http://movielens.org). The data sets were collected over various periods of time, depending on the size of the set.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

### Start Spark Session

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('rec_sys').getOrCreate()

### Import Data

In [4]:
df = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [5]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
df.count()

1501

### Split Data Into Train and Test

In [7]:
train, test = df.randomSplit([0.8, 0.2])

### Collaberative Filtering Using Alternating Least Squares (ALS)

In [8]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [10]:
model=als.fit(train)

### Predictions

In [11]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [12]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     12|    11|
|     16|    11|
|     18|    11|
|     25|    11|
|     38|    11|
|     40|    11|
|     48|    11|
|     51|    11|
|     71|    11|
|     76|    11|
|     79|    11|
|     94|    11|
+-------+------+



In [13]:
recommendation = model.transform(single_user)

In [15]:
recommendation.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     12|    11|  5.1220517|
|     25|    11|  4.8468966|
|     40|    11|   3.596761|
|     71|    11|   3.579913|
|     38|    11|  3.1542044|
|     48|    11|  3.0144176|
|     16|    11|  2.9449656|
|      9|    11|  1.9233412|
|     18|    11|  1.7710524|
|     76|    11|  1.1665272|
|     94|    11|  1.0497217|
|     79|    11| 0.26726684|
|     51|    11|-0.26928842|
+-------+------+-----------+



In [16]:
spark.stop()